In [1]:
import json
import pandas as pd
import random
import torch
from numerical_regressor import TokenNumericCollegeResultsDataset, CombinedDelayedRegressor, mappings

combined_data = {}
uncategorized_data = json.load(open('../../scraping/combined_collegeresults_data.json', 'r'))
standardized_output = json.load(open('../../categorization/standardized_output.json', 'r'))

colleges_list = open('../../categorization/all-colleges.txt').readlines()
colleges_list = [college[:college.index(' (')] for college in colleges_list]

college_data = pd.read_csv('../../categorization/college_acceptance.csv')

/Users/Yourui/Documents/nochances/nochances/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
major_data = pd.read_csv('../../categorization/major_data.csv')
major_data['combined'] = major_data[major_data.columns.drop(['Name', 'Total'])].values.tolist()
major_data = major_data[['Name', 'combined', 'Total']]
def to_frequencies(counts, total):
    return [float(count/total) if total else 0 for count in counts]
major_data['combined'] = major_data.apply(lambda x: to_frequencies(x['combined'], x['Total']), axis=1)

In [3]:
college_data = pd.merge(major_data, college_data, on="Name")
college_data

,Name,combined,Total,Applicants total,Admissions total,Enrolled total,Percent of freshmen submitting SAT scores,Percent of freshmen submitting ACT scores,SAT Critical Reading 25th percentile score,SAT Critical Reading 75th percentile score,...,Percent of freshmen receiving federal grant aid,Percent of freshmen receiving Pell grants,Percent of freshmen receiving other federal grant aid,Percent of freshmen receiving state/local grant aid,Percent of freshmen receiving institutional grant aid,Percent of freshmen receiving student loan aid,Percent of freshmen receiving federal student loans,Percent of freshmen receiving other loan aid,Endowment assets (year end) per FTE enrollment (GASB),Endowment assets (year end) per FTE enrollment (FASB)
0,Alabama A & M University,"[0.1782178217821782, 0.11287128712871287, 0.23...",505,6142.0,5521.0,1104.0,15.0,88.0,370.0,450.0,...,81.0,81.0,7.0,1.0,32.0,89.0,89.0,1.0,NaN,NaN
1,University of Alabama at Birmingham,"[0.10211565585331453, 0.051057827926657265, 0....",3545,5689.0,4934.0,1773.0,6.0,93.0,520.0,640.0,...,36.0,36.0,10.0,0.0,60.0,56.0,55.0,5.0,24136.0,NaN
2,University of Alabama in Huntsville,"[0.3168114460909555, 0.11088400613183444, 0.24...",1957,2054.0,1656.0,651.0,34.0,94.0,510.0,640.0,...,31.0,31.0,4.0,1.0,63.0,46.0,46.0,3.0,11502.0,NaN
3,Alabama State University,"[0.01282051282051282, 0.12307692307692308, 0.2...",390,10245.0,5251.0,1479.0,18.0,87.0,380.0,480.0,...,76.0,76.0,13.0,11.0,34.0,81.0,81.0,0.0,13202.0,NaN
4,The University of Alabama,"[0.13406237343053867, 0.015525853921965708, 0....",7407,30975.0,17515.0,6454.0,23.0,76.0,490.0,620.0,...,20.0,18.0,4.0,3.0,50.0,42.0,41.0,8.0,19469.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261,Providence Christian College,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",21,122.0,65.0,20.0,NaN,NaN,NaN,NaN,...,50.0,50.0,14.0,0.0,100.0,64.0,64.0,14.0,NaN,350.0
1262,Polytechnic University of Puerto Rico-Orlando,"[0.64, 0.04, 0.32, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,100.0,0.0,50.0,0.0,50.0,50.0,0.0,NaN,NaN
1263,University of North Georgia,"[0.0031409501374165686, 0.06949352179034157, 0...",2547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1264,Kennesaw State University,"[0.1752136752136752, 0.14213358657803102, 0.22...",6318,9471.0,5355.0,3194.0,67.0,45.0,500.0,590.0,...,36.0,36.0,2.0,68.0,4.0,51.0,51.0,3.0,1180.0,NaN


In [4]:
residences = [value['residence'] for value in standardized_output.values()]
for i, row in enumerate(open('../../categorization/categorized.json', 'r').readlines()):
    numerical_data = json.loads(row)

selective_enough = college_data.loc[college_data['Admissions total']/college_data['Applicants total'] < .30]['Name'].to_list()
selective_enough

['University of Arkansas at Pine Bluff',
 'California Institute of Technology',
 'University of California-Berkeley',
 'University of California-Los Angeles',
 'Claremont McKenna College',
 'Harvey Mudd College',
 'Pitzer College',
 'Pomona College',
 'University of Southern California',
 'Colorado College',
 'Wesleyan University',
 'Yale University',
 'Georgetown University',
 'Albany State University',
 'Emory University',
 'Chicago State University',
 'University of Chicago',
 'Northwestern University',
 'University of Notre Dame',
 'Tulane University of Louisiana',
 'Bates College',
 'Bowdoin College',
 'Colby College',
 'Johns Hopkins University',
 'Amherst College',
 'Harvard University',
 'Massachusetts Institute of Technology',
 'Tufts University',
 'Wellesley College',
 'Williams College',
 'Carleton College',
 'Mississippi Valley State University',
 'Rust College',
 'Missouri Valley College',
 'College of the Ozarks',
 'Washington University in St Louis',
 'Dartmouth College'

In [5]:
fake_data_index = 0
for i, row in enumerate(open('../../categorization/categorized.json', 'r').readlines()):
    numerical_data = json.loads(row)

    if numerical_data == {'skip': True}:
        continue

    try:
        post_id = list(uncategorized_data.keys())[i]

        selected_data = [
                    numerical_data['basic_info']['ethnicity'],
                    numerical_data['basic_info']['gender'],
                    numerical_data['basic_info']['income_bracket'],
                    numerical_data['basic_info']['gpa'],
                    numerical_data['basic_info']['ap_ib_courses'],
                    numerical_data['basic_info']['ap_ib_scores'],
                    numerical_data['basic_info']['test_score'],
                    numerical_data['basic_info']['location'],
                    numerical_data['basic_info']['legacy'],
                    numerical_data['basic_info']['first_gen'],
                    numerical_data['basic_info']['languages'],
                    numerical_data['basic_info']['special_talents'],
                    numerical_data['basic_info']['hooks'],
                ] + list(numerical_data['ecs'].values()) + list(numerical_data['awards'].values())

        try:
            other_data = standardized_output[post_id]
            results = other_data['results']
            combined_data.update({post_id:{
                'major':other_data['major'],
                'residence':other_data['residence'],
                'ecs':other_data['extracurriculars'],
                'awards':other_data['awards'],
                'numeric':selected_data,
                'results':results
            }})
        except KeyError:
            continue
    
    except IndexError:
        selected_data = [
            numerical_data['basic_info']['ethnicity'],
            numerical_data['basic_info']['gender'],
            numerical_data['basic_info']['income_bracket'],
            random.choices([0, 1, 2, 3, 4], weights=[0.1, 0.3, 0.5, 0.08, 0.02])[0],
            numerical_data['basic_info']['ap_ib_courses'],
            numerical_data['basic_info']['ap_ib_scores'],
            numerical_data['basic_info']['test_score'],
            numerical_data['basic_info']['location'],
            numerical_data['basic_info']['legacy'],
            numerical_data['basic_info']['first_gen'],
            numerical_data['basic_info']['languages'],
            numerical_data['basic_info']['special_talents'],
            numerical_data['basic_info']['hooks'],
        ] + list(numerical_data['ecs'].values()) + list(numerical_data['awards'].values())

        combined_data.update({i: {
            'major': mappings[random.randint(0,11)][0],
            'residence': random.choice(residences),
            'ecs': [],
            'awards': [],
            'numeric': selected_data,
            'results': [{
                'school_name': random.choice(selective_enough),
                'in_state': 0,
                'accepted': 0,
                'round': random.choice([1, 3]),
            } for _ in range(random.randint(1,5))]
        }})
        fake_data_index += 1

In [6]:
len(combined_data)

3081

In [7]:
# dataset = TokenNumericCollegeResultsDataset(combined_data, college_data)
# torch.save(dataset, 'better_stats_actually_numerical_fake_data.pt')
dataset = torch.load('better_stats_actually_numerical_fake_data.pt')

In [8]:
full_data_size = len(dataset)
train_size = int(full_data_size * 0.8)
print(f"Train Data Size: {train_size}")

gen = torch.Generator()
gen.manual_seed(0)

batch_size = 128
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, full_data_size - train_size], generator=gen)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

Train Data Size: 17587


In [9]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
device

device(type='mps')

In [10]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, n_epochs, device):
    best_val_acc = 0
    for epoch in range(n_epochs):
        for param_group in optimizer.param_groups:
            print(f"Current learning rate: {param_group['lr']}")

        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0

        for i, batch in enumerate(train_loader):
            optimizer.zero_grad()

            for k, v in batch.items():
                batch[k] = v.to(device)

            outputs = model(batch['inputs'])
            loss = criterion(outputs, batch['target'])

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            train_loss += loss.item()
            train_total += batch['target'].size(0)
            train_correct += ((torch.sigmoid(outputs) > 0.5) == batch['target']).sum().item()

        train_loss /= len(train_loader)
        train_acc = train_correct / train_total

        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for batch in val_loader:
                for k, v in batch.items():
                    batch[k] = v.to(device)

                outputs = model(batch['inputs'])
                loss = criterion(outputs, batch['target'])

                val_loss += loss.item()
                val_total += batch['target'].size(0)
                val_correct += ((torch.sigmoid(outputs) > 0.5) == batch['target']).sum().item()

            print(torch.sigmoid(outputs))

        val_loss /= len(val_loader)
        val_acc = val_correct / val_total

        scheduler.step(val_loss)

        print(f'Epoch {epoch+1}/{n_epochs}:')
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'reduced_v4_fake_data_numerical.pt')
            #v2 is 81% val acc

    return model

In [11]:
model = CombinedDelayedRegressor().to(device)

n_epochs = 20
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs)
model = train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, n_epochs, device)

Current learning rate: 0.0005
tensor([0.6537, 0.7107, 0.8423, 0.4905, 0.4909, 0.5798, 0.7036, 0.7914, 0.9104,
        0.5818, 0.4412, 0.4045, 0.5930, 0.7851, 0.3922, 0.8242, 0.6403, 0.6595,
        0.5926, 0.4342, 0.3750, 0.5562, 0.3243, 0.4454, 0.7872, 0.6788, 0.6432,
        0.2417, 0.5076, 0.7227, 0.6446, 0.4274, 0.7067, 0.8223, 0.8837, 0.9300,
        0.8275, 0.3214, 0.8403, 0.5833, 0.9392, 0.5023, 0.5153, 0.5004, 0.7400],
       device='mps:0')
Epoch 1/50:
Train Loss: 0.6603, Train Acc: 0.6174
Val Loss: 0.5703, Val Acc: 0.7180
Current learning rate: 0.0004998395258912337


/Users/Yourui/Documents/nochances/nochances/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


tensor([0.6051, 0.7151, 0.4637, 0.5864, 0.4873, 0.3127, 0.7140, 0.5937, 0.5170,
        0.3530, 0.5049, 0.4476, 0.1681, 0.7458, 0.9638, 0.6444, 0.6093, 0.6413,
        0.4157, 0.2249, 0.9089, 0.6137, 0.4491, 0.6020, 0.8793, 0.5602, 0.7824,
        0.5524, 0.1590, 0.6062, 0.6310, 0.5948, 0.8588, 0.7498, 0.2452, 0.4261,
        0.8647, 0.2643, 0.5665, 0.5388, 0.6017, 0.3715, 0.8592, 0.5616, 0.5942],
       device='mps:0')
Epoch 2/50:
Train Loss: 0.5674, Train Acc: 0.7165
Val Loss: 0.5253, Val Acc: 0.7344
Current learning rate: 0.0004998638389010577
tensor([0.2515, 0.8084, 0.5624, 0.7550, 0.5630, 0.2027, 0.9887, 0.5432, 0.8041,
        0.2834, 0.8934, 0.8083, 0.5305, 0.5896, 0.2762, 0.7427, 0.9204, 0.3250,
        0.9537, 0.6069, 0.8631, 0.5812, 0.2379, 0.3677, 0.9922, 0.2993, 0.5799,
        0.5005, 0.1819, 0.8240, 0.8812, 0.7560, 0.6721, 0.5325, 0.6383, 0.5647,
        0.9747, 0.8902, 0.6636, 0.5945, 0.5683, 0.6137, 0.1086, 0.3149, 0.6693],
       device='mps:0')
Epoch 3/50:
Train Loss: